# CatBoost (Gradient-boosting), performance between customer order quantities

In [ ]:
%pip install nbformat pandas numpy matplotlib seaborn scikit-learn catboost 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
import sys
sys.path.append('../../src')

In [ ]:
from data_config import ProductHandler as ph

In [ ]:
ph.custom_prod_set()

In [ ]:
# define the features and target variable from 'product_sales'
# CHANGE THE CUSTOMER CODE HERE >>>>>>>>>>
custom_code_df = pd.DataFrame(ph.get_custom_code_data('FRE'))

# customer code for later referencing
custom_ref = custom_code_df.loc[0, 'ProductNumber'][:3].lower().upper()
print(custom_ref)

custom_code_df.head()

In [ ]:
from app_utils import Transform as trans
from model_utils import *
from sklearn.pipeline import Pipeline

In [ ]:
# Compute OrderQuantity values with a zscore with threshold of 3 and remove them (Outlier removal)
custom_code_df = trans.compute_zscore(custom_code_df)

custom_code_df.head()

In [ ]:
X = custom_code_df.drop('OrderQuantity', axis=1)
y = custom_code_df.OrderQuantity

In [ ]:
# Trian and transforms data (CatBoost transforms natively so might not need to transform)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train_preprocessed, X_val_preprocessed = trans.transform_data(X_train, X_val, X)

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
catb_params = find_best_hyperparameters(CatBoostRegressor(), param_grids(CatBoostRegressor().__class__.__name__), X_train, y_train)

In [ ]:
# Train and get metric scores of CatBoost with best parameters
catb_params = CatBoostRegressor(**catb_params)

catb_params.fit(X_train, y_train)

print('----------- TRAINING METRICS -----------')
train_metrics = evaluate_model(catb_params, X_train, y_train)
print('\n\n')
print('----------- TESTING METRICS -----------')
test_metrics = evaluate_model(catb_params, X_val, y_val)

In [ ]:
# Get model predictions
y_train_pred = catb_params.predict(X_train)
y_val_pred = catb_params.predict(X_val)

In [ ]:
# Create a DetailedOrderDate column using week and weekday information.
custom_code_df['DetailedOrderDate'] = pd.to_datetime(
    custom_code_df['order_year'].astype(str) +
    custom_code_df['order_week'].astype(str).str.zfill(2) +
    custom_code_df['order_weekday'].astype(str),
    format='%Y%W%w'
)

In [ ]:
# Create subplots (2 rows, 2 columns: one for line plot, one for residual plot)
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18, 12)) 
fig.subplots_adjust(hspace=0.5, wspace=0.4)

# Time Series Comparison
sns.lineplot(x=custom_code_df.loc[y_val.index, 'OrderDate'], y=y_val, label='Actual', ax=axes[0, 0])
sns.lineplot(x=custom_code_df.loc[y_val.index, 'OrderDate'], y=y_val_pred, label='Predicted', ax=axes[0, 0])
axes[0, 0].set_title(f'{custom_ref} Neural Network - Time Series', fontsize=16)
axes[0, 0].set_xlabel('Date', fontsize=14)  
axes[0, 0].set_ylabel('Order Quantity', fontsize=14)

# Residual Plot
residuals = y_val - y_val_pred
sns.scatterplot(x=y_val_pred, y=residuals, alpha=0.6, ax=axes[0, 1])
axes[0, 1].axhline(0, color='r', linestyle='--')
axes[0, 1].set_title(f'{custom_ref} Neural Network - Residuals', fontsize=16)  
axes[0, 1].set_xlabel('Predicted Values', fontsize=14)  
axes[0, 1].set_ylabel('Scaled Residuals', fontsize=14)

# Actual vs Predicted Scatter Plot
min_val = min(y_val.min(), y_val_pred.min())
max_val = max(y_val.max(), y_val_pred.max())
sns.scatterplot(x=y_val, y=y_val_pred, alpha=0.6, ax=axes[1, 0], label='Predicted')
axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=1)  # Reference line
axes[1, 0].set_title(f'{custom_ref} Neural Network - Accuracy', fontsize=16)
axes[1, 0].set_xlabel('Actual Values', fontsize=14)
axes[1, 0].set_ylabel('Predicted Values', fontsize=14)  
axes[1, 0].legend(fontsize=12)

# Monthly Trend Comparison
monthly_data = custom_code_df[['order_month']].loc[custom_code_df.index.intersection(X_val.index)].copy()
monthly_data['Actual'] = y_val
monthly_data['Predicted'] = y_val_pred

sns.lineplot(x='order_month', y='Predicted', data=monthly_data, label='Predicted', ax=axes[1, 1])
sns.lineplot(x='order_month', y='Actual', data=monthly_data, label='Actual', ax=axes[1, 1], color='black', linestyle='--')
axes[1, 1].set_title(f'{custom_ref} Neural Network - Monthly Trend Comparison (2022-2025)', fontsize=16)  
axes[1, 1].set_xlabel('Month', fontsize=14)
axes[1, 1].set_ylabel('Order Quantity', fontsize=14)

# Adjust x-tick labels
plt.setp(axes[0, 0].get_xticklabels(), rotation=45, ha='right', fontsize=12)
plt.setp(axes[1, 1].get_xticklabels(), rotation=45, ha='right', fontsize=12)

plt.tight_layout()
plt.show()